In [1]:
# Untuk pengolahan data
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path

# Untuk visualisasi data
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_palette('Set1')
sns.set()

# Untuk pemodelan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Untuk menghilangkan warnings saat plotting seaborn
import warnings
warnings.filterwarnings('ignore')

# Untuk mengupload file
import os

In [2]:
rating = pd.read_csv('/home/rating_baru.csv')

In [3]:
rating.head(2)

,User_Id,Place_Id,Place_Ratings
0,1,438,2
1,20,438,2


In [9]:
import os
os.listdir("/content")


['.config',
 'recommender_model_weights.h5',
 'recommender_model.zip',
 'sample_data']

In [10]:
import zipfile
import os

# Nama file zip yang telah diunggah
zip_filename = "recommender_model.zip"

# Path untuk mengekstrak
extract_path = "/content"

# Mengekstrak file zip
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Menampilkan isi dari folder yang telah diekstrak
print("Isi folder yang telah diekstrak:")
print(os.listdir(extract_path))


Isi folder yang telah diekstrak:
['.config', 'recommender_model_weights.h5', 'recommender_model', 'recommender_model.zip', 'sample_data']


In [14]:
model = tf.keras.models.load_model("/content/recommender_model")
place = pd.read_csv('/content/place_malang.csv')

In [16]:
# Menyiapkan dataframe
#place_df = place[['Place_Id','Place_Name','Category','Rating','Price']]
#place_df.columns = ['id','place_name','category','rating','price']
#df = rating.copy()

In [17]:
# Mengambil sample user
#user_id = df.User_Id.sample(1).iloc[0]
#place_visited_by_user = df[df.User_Id == user_id]

In [25]:
df = rating.copy()
df.head()

,User_Id,Place_Id,Place_Ratings
0,1,438,2
1,20,438,2
2,39,438,3
3,58,438,2
4,77,438,5


MELAKUKAN ENCODING


1. MEMBUAT FUNGSI ENCODING

In [26]:
def dict_encoder(col, data=df):

  # Mengubah kolom suatu dataframe menjadi list tanpa nilai yang sama
  unique_val = data[col].unique().tolist()

  # Melakukan encoding value kolom suatu dataframe ke angka
  val_to_val_encoded = {x: i for i, x in enumerate(unique_val)}

  # Melakukan proses encoding angka ke value dari kolom suatu dataframe
  val_encoded_to_val = {i: x for i, x in enumerate(unique_val)}
  return val_to_val_encoded, val_encoded_to_val

2. Encoding dan Mapping Kolom User

In [27]:
# Encoding User_Id
user_to_user_encoded, user_encoded_to_user = dict_encoder('User_Id')

# Mapping User_Id ke dataframe
df['user'] = df['User_Id'].map(user_to_user_encoded)

3. Encoding dan Mapping Kolom Place

In [28]:
# Encoding Place_Id
place_to_place_encoded, place_encoded_to_place = dict_encoder('Place_Id')

# Mapping Place_Id ke dataframe place
df['place'] = df['Place_Id'].map(place_to_place_encoded)

In [29]:
# Mendapatkan jumlah user dan place
num_users, num_place = len(user_to_user_encoded), len(place_to_place_encoded)

# Mengubah rating menjadi nilai float
df['Place_Ratings'] = df['Place_Ratings'].values.astype(np.float32)

# Mendapatkan nilai minimum dan maksimum rating
min_rating, max_rating = min(df['Place_Ratings']), max(df['Place_Ratings'])

print(f'Number of User: {num_users}, Number of Place: {num_place}, Min Rating: {min_rating}, Max Rating: {max_rating}')

Number of User: 300, Number of Place: 57, Min Rating: 1.0, Max Rating: 5.0


In [30]:
# Mengacak dataset
df = df.sample(frac=1, random_state=42)
df.head(2)

,User_Id,Place_Id,Place_Ratings,user,place
203,258,476,2.0,203,38
266,232,488,3.0,266,50


 Menyiapkan DataFrame untuk Menampilkan Hasil Rekomendasi

In [31]:
place_df = place[['Place_Id','Place_Name','Category','Rating','Price']]
place_df.columns = ['id','place_name','category','rating','price']
df = rating.copy()

Menyiapkan contoh User untuk Menampilkan Rekomendasi

MENCARI USER

In [32]:
# Mengambil sample user
user_id = df.User_Id.sample(1).iloc[0]
place_visited_by_user = df[df.User_Id == user_id]

Mencari Lokasi yang belum Dikunjungi User

In [33]:
# Membuat data lokasi yang belum dikunjungi user
place_not_visited = place_df[~place_df['id'].isin(place_visited_by_user.Place_Id.values)]['id']
place_not_visited = list(
    set(place_not_visited)
    .intersection(set(place_to_place_encoded.keys()))
)

place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_place_array = np.hstack(
    ([[user_encoder]] * len(place_not_visited), place_not_visited)
)

Menampilkan Hasil Rekomendasi untuk User

In [35]:
# Mengambil top 7 recommendation
ratings = model.predict(user_place_array).flatten()
top_ratings_indices = ratings.argsort()[-7:][::-1]
recommended_place_ids = [
    place_encoded_to_place.get(place_not_visited[x][0]) for x in top_ratings_indices
]

print('Daftar rekomendasi untuk: {}'.format('User ' + str(user_id)))
print('===' * 15,'\n')
print('----' * 15)
print('Tempat dengan rating wisata paling tinggi dari user')
print('----' * 15)

top_place_user = (
    place_visited_by_user.sort_values(
        by = 'Place_Ratings',
        ascending=False
    )
    .head(5)
    .Place_Id.values
)

place_df_rows = place_df[place_df['id'].isin(top_place_user)]
for row in place_df_rows.itertuples():
    print(row.place_name, ':', row.category)

print('')
print('----' * 15)
print('Top 7 place recommendation')
print('----' * 15)

recommended_place = place_df[place_df['id'].isin(recommended_place_ids)]
for row, i in zip(recommended_place.itertuples(), range(1,8)):
    print(i,'.', row.place_name, '\n    ', row.category, ',', 'Harga Tiket Masuk ', row.price, ',', 'Rating Wisata ', row.rating,'\n')

print('==='*15)

2/2 [==============================] - 0s 5ms/step
Daftar rekomendasi untuk: User 265

------------------------------------------------------------
Tempat dengan rating wisata paling tinggi dari user
------------------------------------------------------------
Pantai Bajulmati : Pantai

------------------------------------------------------------
Top 7 place recommendation
------------------------------------------------------------
1 . Bukit Teletubbies Bromo 
     Alam , Harga Tiket Masuk  nan , Rating Wisata  4.6 

2 . Sumber Maron 
     Alam, Pemandian , Harga Tiket Masuk  nan , Rating Wisata  4.3 

3 . Masjid Tiban Turen 
     Religi , Harga Tiket Masuk  nan , Rating Wisata  4.4 

4 . Pantai Batu Bengkung 
     Pantai , Harga Tiket Masuk  nan , Rating Wisata  4.5 

5 . Pantai Kondang Merak 
     Pantai , Harga Tiket Masuk  nan , Rating Wisata  4.5 

6 . Pantai banyu Anjlok 
     Pantai, Alam , Harga Tiket Masuk  nan , Rating Wisata  4.6 

7 . Pantai Tamban 
     Pantai , Harga Tik